# คุณสมบัติของช่องสัญญาณและผลกระทบที่เกิดกับสัญญาณ

## วัตถุประสงค์
1. เข้าใจคุณสมบัติของช่องสัญญาณแบบ low-pass channel และ band-pass channel
1. เข้าใจผลที่เกิดขึ้นจากการส่งสัญญาณแบบดิจิทัลผ่านช่องสัญญาณแบบ low-pass และ band-pass
1. เข้าใจความสัมพันธ์ระหว่างแบนด์วิธด์แบบดิจิทัลและแบบแอนะล็อก

## การเตรียมตัว
เรียกใช้โมดูล `sigproc` รวมถึงอิมพอร์ทชื่อ `bkp` ออกมาจากโมดูลเพื่อเรียกใช้งานไลบรารี Bokeh สำหรับสร้างกราฟโดยตรง

In [ ]:
from sigproc import Signal,start_notebook
bkp = start_notebook()

# คอนฟิกสำหรับใช้กำหนดคุณสมบัติให้กราฟโดเมนเวลา
TIME_FIG = dict(height=300,x_axis_label="Time (ms)",y_axis_label="Voltage (V)")
TX_LINE = dict(color="blue",line_width=3)
RX_LINE = dict(color="red",line_width=1)

เตรียมฟังก์ชัน `create_ttl_signal` เพื่อสร้างสัญญาณดิจิทัลแบบ TTL ตามบิตเรทที่กำหนด

In [ ]:
def create_ttl_signal(data,bit_rate):
    data = data.replace(" ","")
    bit_interval = 1/bit_rate
    sig = Signal(duration=len(data)/bit_rate)
    for i,bit in enumerate(data):
        level = {"0":0,"1":5}[bit]
        sig.add_pulse(value=level,time=i*bit_interval,duration=bit_interval)
    return sig

## สร้างและวิเคราะห์สัญญาณดิจิทัล
นำฟังก์ชัน `create_ttl_signal` มาทดลองสร้างสัญญาณ และแสดงสัญญาณที่ได้ในโดเมนเวลาและโดเมนความถี่

In [ ]:
DATA = "01010010"
BIT_RATE = 10  # bps

s1 = create_ttl_signal(data=DATA,bit_rate=BIT_RATE)
s1.plot_time()
s1.plot_freq(fig_options=dict(x_range=(-20,1000))) # ซูมให้เห็นเฉพาะความถี่ไม่เกิน 1000 Hz

สเปกตรัมความถี่ของสัญญาณแสดงให้เห็นคุณสมบัติที่สำคัญของสัญญาณแบบดิจิทัลดังนี้
* พลังงานส่วนใหญ่กองอยู่ในย่านความถี่ต่ำ
* ประกอบไปด้วยความถี่ทุกย่านตั้งแต่ศูนย์ไปถึงอนันต์

## ช่องสัญญาณแบบ Low-Pass
ช่องสัญญาณแบบ low-pass ยอมให้สัญญาณย่านความถี่ต่ำผ่านได้แต่ความถี่สูงผ่านไม่ได้
<img src="pics/low-pass.png">
ลองนำสัญญาณ `s1` ข้างต้นมาวิเคราะห์ผลกระทบเมื่อถูกส่งผ่านช่องสัญญาณแบบ low-pass ที่กำหนดให้ความถี่สูงสุดที่ผ่านได้คือ `FMAX` จะเห็นได้ว่าสัญญาณที่มีความถี่สูงกว่าที่กำหนดไว้จะถูกกำจัดทิ้งทั้งหมด ซึ่งทำให้ได้สัญญาณในโดเมนเวลาที่เปลี่ยนไป (แสดงด้วยสีแดง ซ้อนทับกับสัญญาณดั้งเดิมที่เป็นสีน้ำเงิน)

หมายเหตุ: ทดลองเปลี่ยนค่า `FMAX` และสังเกตรูปร่างของสัญญาณที่เปลี่ยนไปทั้งในโดเมนเวลาและโดเมนความถี่

In [ ]:
FMAX = 100
s1_lowpass = s1.copy()
s1_lowpass.filter((0,FMAX))
s1_lowpass.plot_freq(fig_options=dict(x_range=(-100,1000)))

# สร้าง figure ด้วย Bokeh เพื่อให้พล็อตสัญญาณซ้อนทับลงไปบนกราฟเดียวกันได้
fig = bkp.figure(**TIME_FIG)
s1_lowpass.plot_time(fig,line_options=RX_LINE)
s1.plot_time(fig,line_options=TX_LINE)
fig.segment(0,0.5,1000,0.5,color="green")  # แสดงขอบเขต TTL logic level 0
fig.segment(0,2.7,1000,2.7,color="green")  # แสดงขอบเขต TTL logic level 1
bkp.show(fig)

## ช่องสัญญาณแบบ Band-Pass
ช่องสัญญาณแบบ band-pass ยอมให้สัญญาณผ่านได้เฉพาะย่านความถี่ที่กำหนด ความถี่ที่ต่ำเกินไปหรือสูงเกินไปไม่สามารถผ่านได้
<img src="pics/band-pass.png">
ลองนำสัญญาณ `s1` เดิมมาวิเคราะห์ผลกระทบเมื่อถูกส่งผ่านช่องสัญญาณแบบ band-pass ที่จำกัดให้ความถี่ผ่านได้เพียงในย่าน `FMIN` ถึง `FMAX`

In [ ]:
FMIN,FMAX = 50,100
s1_bandpass = s1.copy()
s1_bandpass.filter((FMIN,FMAX))
s1_bandpass.plot_freq(fig_options=dict(x_range=(-100,1000)))

# สร้าง figure ด้วย Bokeh เพื่อให้พล็อตสัญญาณซ้อนทับลงไปบนกราฟเดียวกันได้
fig = bkp.figure(**TIME_FIG)
s1_bandpass.plot_time(fig,line_options=RX_LINE)
s1.plot_time(fig,line_options=TX_LINE)
bkp.show(fig)

เห็นได้ว่าสัญญาณดิจิทัลต้องการช่องสัญญาณแบบ low-pass และไม่สามารถส่งผ่านลงไปตรง ๆ ในช่องสัญญาณแบบ band-pass ได้

## ความสัมพันธ์ระหว่างบิตเรทและแบนด์วิธด์
Nyquist พิสูจน์ความสัมพันธ์ระหว่าง digital bandwidth (bitrate) และ analog bandwidth ดังสมการ

$$bitrate = 2 \times B \times \log_2(L)$$

โดยที่
* $bitrate$ แทนอัตราการส่งข้อมูลในหน่วยบิตต่อวินาที
* $B$ แทนแบนด์วิธด์ของช่องสัญญาณในหน่วยเฮิร์ตซ์
* $L$ แทนจำนวนระดับสัญญาณดิจิทัลที่เป็นไปได้ ซึ่งการส่งสัญญาณแบบ TTL มีค่า L=2 เนื่องจากใช้สัญญาณเพียงสองระดับคือ 0 โวลท์ และ 5 โวลท์

ทดสอบทฤษฎีบทของ Nyquist กับโค้ดด้านล่าง โดยลองปรับเปลี่ยนข้อมูล `DATA` ให้อยู่ในรูปที่ใช้ความถี่ต่ำที่สุดและสูงที่สุด และปรับเปลี่ยน `BIT_RATE` และ `BANDWIDTH` เพื่อสังเกตผลกระทบที่เกิดขึ้นกับสัญญาณที่ฝั่งรับ

In [ ]:
#DATA = "1111 1111 1111 1111" # ใช้ความถี่ต่ำที่สุด
DATA = "0101 0101 0101 0101" # ใช้ความถี่สูงที่สุด
#DATA = "0000 1111 0101 0101"
BIT_RATE = 30
BANDWIDTH = 15

fig = bkp.figure(**TIME_FIG)

s = create_ttl_signal(data=DATA,bit_rate=BIT_RATE)
s.plot_time(fig=fig,line_options=TX_LINE)
s.filter((0,BANDWIDTH))
s.plot_time(fig=fig,line_options=RX_LINE)
bkp.show(fig)